In [13]:
# db = int(input('Database ID (2 for 4 chamber and 17 for short axis): '))
# basedir = input('Base directory (e.g. D:/ML_data/PAH): ')
# scale = int(input('Scale (16, 8, 4, or -1): '))
# mask_id = int(input('Mask ID (1-5): '))
# level = int(input('Preprocessing level (1-4): '))

In [1]:
import os
import h5py
import numpy as np
import pandas as pd

In [50]:
def check_dir(dir_):
    """
    Check the existence of a given directory.
    If not exist, create the directory

    Args:
        dir_ (str): full directory path

    """
    if not os.path.exists(dir_):
        os.mkdir(dir_)

### Step 0: Converting .mat file to .npy (ndarray) file

In [2]:
def mat2npy(data_path, db, out_dir):
    """Converting mat file to npy file

    Args:
        data_path (str): full path of .mat file,
        db (int): data base id, 2 for 4 chamber and 17 for short axis
        out_dir (str): the directory for saving output file
    """
    print('Converting data to ndarray')
    # fname = 'PAH1DB%s.mat' % db
    # data_path = os.path.join(basedir, fname)
    f = h5py.File(data_path, 'r')

    data = f['data'][()].transpose()
    check_dir(out_dir)
    out_path = os.path.join(out_dir, 'PAH1DB%s.npy' % db)
    np.save(out_path, data)
    # data_ = torch.from_numpy(data).to_sparse()
    # out_path = os.path.join(basedir, 'RegPAH1DB%s.hdf5' % db)
    # f = h5py.File(out_path, "w")
    # dest = f.create_dataset()

    labels = f['labels'][()].reshape(-1)
    # max_dist = f['maxDists'][()].reshape(-1)
    # df = pd.DataFrame(data={'Label': labels, 'Max dist': max_dist})
    df = pd.DataFrame(data={'Label': labels})
    csv_path = os.path.join(out_dir, 'info_DB%s.csv' % db)
    df.to_csv(csv_path)
    print('Completed!')

In [3]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

interact_manual(mat2npy, db=widgets.Dropdown(
    options=[('Four chamber', 2), ('Short axis',17)],
    value=2,
    description='Database',
    disabled=False,
), 
                data_path = input('Data full path (e.g. D:/ML_data/PAH/PAH1DB2.mat): '), 
                out_dir = input('Where to save the output (e.g. D:/ML_data/PAH): '))


Data full path (e.g. D:/ML_data/PAH/PAH1DB2.mat): D:/ML_data/PAH/PAH1DB2.mat
Where to save the output (e.g. D:/ML_data/PAH): D:/ML_data/PAH


interactive(children=(Text(value='D:/ML_data/PAH/PAH1DB2.mat', description='data_path'), Dropdown(description=…

<function __main__.mat2npy(data_path, db, out_dir)>

## Step 1: Registration

In [4]:
# def load_data(basedir, db):
#     data_path = os.path.join(basedir, 'PAH1DB%s.npy' % db)
#     data = np.load(data_path)
#     return data

def load_landmark(reg_fpath, db):
    """Load landmark location file

    Args:
        reg_fpath (str): full path to landmark location .xlsx file
        db (int): data base id, 2 for 4 chamber and 17 for short axis
    Returns:
        landmark locations, pandas dataframe, shape (n_sample, 2 + n_landmark * 2)
    """
#     reg_fpath = os.path.join(basedir, 'regMRI4ChFull.xlsx')
    if db == 2:
        sheet_name = '4ch'
        # sheet_name = 0
        col_names = ['ID', 'Group', 'mitral ann X', 'mitral ann Y',
                     'LVEDV apex X', 'LVEDV apex Y', 'Spinal X', 'Spinal Y']
    elif db == 17:
        sheet_name = 'SA'
        col_names = ['ID', 'Group', 'inf insertion point X', 'insertion point Y',
                     'sup insertion point X', 'sup insertion point Y', 'RV inf X', 'RV inf Y']

    reg_df = pd.read_excel(reg_fpath, sheet_name=sheet_name, usecols=col_names)
    
    return reg_df

In [8]:
import sys
sys.path.append('..')
from kale.prepdata.prep_cmr import regMRI

def proc_reg(db, data_path, regf_path, out_dir, sample_id=1007):
    """Aligning all images towards a target sample and save the output 
    to the given directory as .npy file
    
    Args:
        db (int): data base id, 2 for 4 chamber and 17 for short axis
        data_path (str): full path to data file
        regf_path (str): full path to landmark location .xlsx file
        out_dir (str): directory for saving output data
        sample_id (int): target sample id for aligning landmarks

    """
    print('Performing registration...')
    # data = load_data(basedir, db)
    data = np.load(data_path)
    reg_df = load_landmark(regf_path, db)

    reg_id = np.where(reg_df['ID'] == sample_id)[0][0]
    data_reg, max_dist = regMRI(data, reg_df, reg_id)
    check_dir(out_dir)
    out_path = os.path.join(out_dir, 'RegPAH1DB%s.npy' % db)
    np.save(out_path, data_reg)

    info_file = os.path.join(out_dir, 'info_DB%s.csv' % db)
    if os.path.exists(info_file):
        info_df = pd.read_csv(info_file, index_col=0)
    else:
        info_df = pd.DataFrame(data={'Label': reg_df['Group'].values})
    info_df['ID'] = reg_df['ID']
    info_df['Max Dist'] = max_dist
    info_df.to_csv(info_file, columns=['ID', 'Label', 'Max Dist'], index=False)

    print('Registration Completed')

In [9]:
from ipywidgets import interact_manual
import ipywidgets as widgets

interact_manual(proc_reg, db=widgets.Dropdown(
    options=[('Four chamber (2)', 2), ('Short axis (17)',17)],
    value=2,
    description='Database',
    disabled=False,), 
                data_path = input('Data full path (e.g. D:/ML_data/PAH/PAH1DB2.npy): '), 
                regf_path = input('Registration file full path (e.g. D:/ML_data/PAH/regMRI4ChFull.xlsx): '), 
                out_dir = input('Where to save the output (e.g. D:/ML_data/PAH/DB2): '),
                sample_id=int(input('Target sample ID used for regisitration (1007): '))
               )

Data full path (e.g. D:/ML_data/PAH/PAH1DB2.npy): D:/ML_data/PAH/PAH1DB2.npy
Registration file full path (e.g. D:/ML_data/PAH/regMRI4ChFull.xlsx): D:/ML_data/PAH/regMRI4ChFull.xlsx
Where to save the output (e.g. D:/ML_data/PAH/DB2): 
Target sample ID used for regisitration (1007): 1007


interactive(children=(Dropdown(description='Database', options=(('Four chamber (2)', 2), ('Short axis (17)', 1…

<function __main__.proc_reg(db, data_path, regf_path, out_dir, sample_id=1007)>

## Step 2: Rescaling

In [10]:
import sys
sys.path.append('..')
from kale.prepdata.prep_cmr import rescale_cmr

def proc_rescale(db, data_path, out_dir, scale=-1):
    """Rescaling data and save the output to the given directory as .npy file

    Args:
        db (int): data base id, 2 for 4 chamber and 17 for short axis
        data_path (str): full path to data file
        out_dir (str): directory for saving output data
        scale (int): scale factor, e.g. the output will be 1/4 of given data if scale=4

    """
    # data_path = os.path.join(basedir, 'RegPAH1DB%s.npy' % db)
    data = np.load(data_path)
    # out_dir = os.path.join(basedir, 'DB%s' % db)
    print('Rescaling data ...')
    check_dir(out_dir)

    if scale == -1:
        for scale_ in [16, 8, 4]:
            print('Scale: 1/%s' % scale_)
            data_ = rescale_cmr(data, scale=scale_)
            out_path = os.path.join(out_dir, 'NoPrs%sDB%s.npy' % (scale_, db))
            np.save(out_path, data_)
    else:
        print('Scale: 1/%s' % scale)
        data_ = rescale_cmr(data, scale=scale)
        out_path = os.path.join(out_dir, 'NoPrs%sDB%s.npy' % (scale, db))
        np.save(out_path, data_)

    print('Completed!')

In [11]:
from ipywidgets import interact_manual
import ipywidgets as widgets

interact_manual(proc_rescale, db=widgets.Dropdown(
    options=[('Four chamber (2)', 2), ('Short axis (17)',17)],
    value=2,
    description='Database',
    disabled=False,), 
                data_path = input('Data full path (e.g. D:/ML_data/PAH/DB2/RegPAH1DB2.npy): '), 
                out_dir = input('Where to save the output (e.g. D:/ML_data/PAH/DB2): '),
                scale = widgets.Dropdown(
                    options=[('16', 16), ('8', 8), ('4', 4), ('-1 (All of above)', -1)],
                    value=4,
                    description='Scale',
                    disabled=False,)
        )

Data full path (e.g. D:/ML_data/PAH/DB2/RegPAH1DB2.npy): D:/ML_data/PAH/DB2/RegPAH1DB2.npy
Where to save the output (e.g. D:/ML_data/PAH/DB2): D:/ML_data/PAH/DB2


interactive(children=(Dropdown(description='Database', options=(('Four chamber (2)', 2), ('Short axis (17)', 1…

<function __main__.proc_rescale(db, data_path, out_dir, scale=-1)>

## Step 3: Preprocessing

In [17]:
import sys
sys.path.append('..')
from kale.prepdata.prep_cmr import cmr_proc


def preproc_(db, data_path, scale, mask_path, mask_id, out_dir, level=1):
    """Preprocessing data of a particular scale with given mask and save
    the output to the given directory as .npy file

    Args:
        db (int): data base id, 2 for 4 chamber and 17 for short axis
        data_path (str): full path to data file
        scale (int): data scale
        mask_path (str): full path to mask file
        mask_id (int): the index of mask to be used for preprocessing
        out_dir (str): directory for saving output data
        level (int): preprocessing level

    """
    print('Preprocessing data ...')
    # print('db:', db, type(db))
    # print('scale:', scale, type(scale))
    # print('maskid:', mask_id, type(mask_id))
    # print('level:', level, type(level))
    fname = 'PrS%sM%sL%sDB%s.npy' % (scale, mask_id, level, db)
    # print(fname)
    data_proc = cmr_proc(data_path, db, scale, mask_path, mask_id, level)
    out_path = os.path.join(out_dir, fname)
    np.save(out_path, data_proc)
    print('Completed')


interact_manual(preproc_, db=widgets.Dropdown(
    options=[('Four chamber (2)', 2), ('Short axis (17)',17)],
    value=2,
    description='Database',
    disabled=False,), 
                data_path = input('Data full path (e.g. D:/ML_data/PAH/DB2/NoPrs4DB2.npy): '), 
                scale=widgets.Dropdown(
                    options=[('16', 16), ('8', 8), ('4', 4), ('-1 (All of above)', -1)],
                    value=4,
                    description='Scale',
                    disabled=False,),
                mask_path = input('Registration file full path (e.g. D:/ML_data/PAH/Prep/AllMasks.mat): '), 
                mask_id=widgets.Dropdown(
                    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4), ('5', 5), ('6', 6), ('7', 7), ('8', 8)],
                    value=5,
                    description='Mask ID:',
                    disabled=False,),
                out_dir = input('Where to save the output (e.g. D:/ML_data/PAH/DB2/PrepData): '),
                level=widgets.Dropdown(
                    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4)],
                    value=1,
                    description='Preprocssing level:',
                    disabled=False,),)

Data full path (e.g. D:/ML_data/PAH/DB2/NoPrs4DB2.npy): D:/ML_data/PAH/DB2/NoPrs4DB2.npy
Registration file full path (e.g. D:/ML_data/PAH/Prep/AllMasks.mat): D:/ML_data/PAH/Prep/AllMasks.mat
Where to save the output (e.g. D:/ML_data/PAH/DB2/PrepData): D:/ML_data/PAH/DB2/PrepData


interactive(children=(Dropdown(description='Database', options=(('Four chamber (2)', 2), ('Short axis (17)', 1…

<function __main__.preproc_(db, data_path, scale, mask_path, mask_id, out_dir, level=1)>

# Classification

In [38]:
import sys
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler


# default_grid = [
#     {'select__estimator__C': np.logspace(-2, 2, 5)},
#     {'clf__C': np.logspace(-3, 2, 6), 'clf__kernel': ['linear']},
#     {'clf__C': np.logspace(-3, 2, 6), 'clf__gamma': np.logspace(-4, -1, 3),
#      'clf__kernel': ['rbf']},
#     ]
class Linear_clf(BaseEstimator, TransformerMixin):
    """Linear classifiers

    Args:
        clf (str): linear classifier to train, options: SVC|LR|Ridge, where
            'SVC' for support vector classifier (linear kernle)
            'LR' for logistic regression
            'Ridge' for ridge classifier (l2 norm)
        n_feat (int): number of discriminative features to select (by LDA)
        cv (int or sklearn cross validation class): cross validation setting
        n_split (int): number of train, test splits, applies only when cv is None
        test_size (float): 
        normalise (bool):
        n_jobs (int):
    """
    def __init__(self, clf='SVC', n_feat=None, cv=None, n_split=10, test_size=0.2, normalise=True, n_jobs=1):
        param_grids = {'SVC': {'clf__C': np.logspace(-3, 2, 6), 'clf__kernel': ['linear']},
                       'LR': {'clf__C': np.logspace(-3, 2, 6)},
                       'Ridge': {'clf__alpha': np.logspace(-3, 2, 6)}}
        self.scaler = StandardScaler()
        self.lda = LDA()
        self.param_grid = param_grids[clf]
        if clf == 'SVC':
            # _clf = Pipeline([('select', SelectFromModel(estimator=LogisticRegression(penalty='l1', solver='liblinear'))),
            #                  ('clf', SVC(max_iter=10000, probability=True))])
            _clf = Pipeline([('clf', SVC(probability=True, max_iter=10000))])
        elif clf == 'LR':
            _clf = Pipeline([('clf', LogisticRegression(max_iter=10000))])
        elif clf == 'Ridge':
            _clf = Pipeline([  # ('scaler', StandardScaler()),
                             ('clf', RidgeClassifier(probability=True, max_iter=10000))])
        else:
            print('Invalid Classifier')
            sys.exit()
        self.n_feat = n_feat
        self.normalise = normalise
        if cv is None:
            cv = StratifiedShuffleSplit(n_splits=n_split, test_size=test_size,
                                        train_size=1 - test_size, random_state=144)
        # self.search = GridSearchCV(_clf, param_grid, n_jobs=n_jobs, cv=cv, iid=False)
        self.search = GridSearchCV(_clf, self.param_grid, n_jobs=n_jobs, cv=cv)

    def fit(self, X, y):
        """Fit a linear classifier with the given data X and labels y

        Args:
            X (array-like): input training data, shape (n_sample, n_feature)
            y (array-like): data labels, shape (n_sample, )

        Returns:
            self
        """
        if self.normalise:
            X = self.scaler.fit_transform(X)
        if self.n_feat is not None:
            self.lda.fit(X, y)
            self.feat_idx = np.abs(self.lda.coef_).argsort()[-self.n_feat:][::-1]  # indices of features with top n_feat fisher coefficients
            X = X[:, self.feat_idx]
        self.search.fit(X, y)
        self.clf = self.search.best_estimator_
        self.clf.fit(X, y)

        return self

    def predict(self, X):
        """Predict the labels of the given data

        Args:
            X (array-like): data for prediction, shape (n_sample_, n_feature)

        Returns:
            Predicted labels, array-like, shape (n_sample_, )

        """
        self.normlise_select(X)
        return self.clf.predict(X)
    
    def predict_proba(self, X):
        """Probability of each class for the given data

        Args:
            X (array-like): data for prediction, shape (n_sample_, n_feature)

        Returns:
            array-like, array-like, shape (n_sample_, n_class)
        """
        self.normlise_select(X)
        return self.clf.predict_proba(X)
    
    def normlise_select(self, X):
        """Preprocess input data, if self.normalise == True, or
        feature selection (self.n_feat) is not None

        Args:
            X (array-like): input data, shape (n_sample, n_feature)

        Returns:
            Preprocessed data, shape (n_sample, n_feature) if self.n_feat is
            not None, otherwise (n_sample, self.n_feat).
            Unchanged if self.normalise == False and self.n_feat is None
        """
        if self.normalise:
            X = self.scaler.transform(X)
        if self.n_feat is not None:
            X = X[:, self.feat_idx]
        
        return X

In [31]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorly.base import fold, unfold


def label_binarizer(y):
    """Converting labels (with subtype) to binary (health control vs patient)

    Args:
        y (array-like): labels, shape (n_sample, )

    Returns:
        binary labels (0 and 1), shape (n_sample, )
    """
    y_ = np.zeros(y.shape)
    y_[np.where(y != 0)] = 1
    return y_


def evaluate_(X, y, kfold=10, random_state=144, return_auc=True):
    """Evaluating linear classifier on given dataset

    Args:
        X (array-like): input data, shape (n_sample, n_feature)
        y (array-like): labels, shape (n_sample, )
        kfold (int): number of splites for k-fold cross validation
        random_state (int): random seed used for random train test spliting
        return_auc (bool): whether compute & report auc in results

    Returns:
        evaluating results, dictionary
    """
    skf = StratifiedKFold(n_splits=kfold, random_state=random_state)
    res = {'fold_accs': [], 'fold_aucs': [], 'acc': None,'auc': None}
    y_pred = np.zeros(y.shape)
    y_dec = np.zeros(y.shape)
    for train, test in skf.split(X, y):
        clf = Linear_clf()
        clf.fit(X[train], y[train])
        y_pred[test] = clf.predict(X[test])
        res['fold_accs'].append(accuracy_score(y[test], y_pred[test]))
        if return_auc:
            y_dec[test] = clf.predict_proba(X[test])[:, 1]
            res['fold_aucs'].append(roc_auc_score(y[test], y_dec[test]))
    res['acc'] = accuracy_score(y, y_pred)
    if return_auc:
        res['auc'] = roc_auc_score(y, y_dec)

    return res

In [32]:
import sys
# sys.path.append('...')
from kale.embed.mpca import MPCA

def main_(data_path, label_path, scale, mask_id, level):
    print('Main Experiemnts for Scale: 1/%s, Mask ID: %s, Processing level: %s' % (scale, mask_id, level))
    # data_path = '%s/DB%s/PrepData' % (basedir, db)
    # fname = 'PrS%sM%sL%sDB%s.npy' % (scale, mask_id, level, db)
    # X = np.load(os.path.join(data_path, fname))
    X = np.load(data_path)
    # info_df = pd.read_csv(os.path.join(basedir, 'info_DB%s.csv' % db))
    info_df = pd.read_csv(label_path)
    y = info_df['Label'].values
    y_ = label_binarizer(y)

    # Perform MPCA dimension reduction
    mpca = MPCA()
    mpca.fit(X)
    Xmpc = mpca.transform(X)
    X_ = unfold(Xmpc, mode=-1).real

    # Evaluating 
    res = evaluate_(X_, y_)

    print('Accuracy:', res['acc'], 'AUC:', res['auc'])

In [39]:
from ipywidgets import interact_manual
import ipywidgets as widgets

interact_manual(main_,
                data_path = input('Data full path (e.g. D:/ML_data/PAH/DB2/PrepData/PrS4M5L1DB2.npy): '),
                label_path = input('Label file full path (e.g. D:/ML_data/PAH/info_DB2.csv)'),
                scale=widgets.Dropdown(
                    options=[('16', 16), ('8', 8), ('4', 4), ('-1 (All of above)', -1)],
                    value=4,
                    description='Scale',
                    disabled=False,),
                mask_id=widgets.Dropdown(
                    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4), ('5', 5), ('6', 6), ('7', 7), ('8', 8)],
                    value=5,
                    description='Mask ID:',
                    disabled=False,),
                level=widgets.Dropdown(
                    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4)],
                    value=1,
                    description='Prep level:',
                    disabled=False,),
               )

Data full path (e.g. D:/ML_data/PAH/DB2/PrepData/PrS4M5L1DB2.npy): D:/ML_data/PAH/DB2/PrepData/PrS4M5L1DB2.npy
Label file full path (e.g. D:/ML_data/PAH/info_DB2.csv)D:/ML_data/PAH/info_DB2.csv


interactive(children=(Dropdown(description='Database', options=(('Four chamber (2)', 2), ('Short axis (17)', 1…

<function __main__.main_(db, data_path, label_path, scale, mask_id, level)>

## Landmark Visulaisation

In [40]:
import tkinter

from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
# Implement the default Matplotlib key bindings.
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np

In [41]:
def sub_img_mark(db, data_path, regf_path, sub, slice_):
    """Get a slice of CMR image and landmark loactions for a subject

    Args:
        db (int): data base id, 2 for 4 chamber and 17 for short axis
        data_path (str): full path to data file
        regf_path (str): full path to landmark location .xlsx file
        sub (int): index of subject
        slice_ (int): index of slice

    Returns:
        image slice, array-like, shape (dim1, dim2)
        landmark loacations, dataframe
    """
    # data = load_data(basedir, db)
    data = np.load(data_path)
    reg_df = load_landmark(regf_path, db)
    
    sub_idx = np.where(reg_df['ID'] == sub)[0][0]
    sub_img = data[..., slice_, sub_idx]
    land_marks = reg_df.iloc[sub_idx, 2:]
    
    return sub_img, land_marks

### Display landmarks

In [42]:
def disp_mark(db, data_path, regf_path, sub, slice_):
    """Display a slice of CMR image with landmark locations

    Args:
        db (int): data base id, 2 for 4 chamber and 17 for short axis
        data_path (str): full path to data file
        regf_path (str): full path to landmark location .xlsx file
        sub (int): index of subject
        slice_ (int): index of slice
        
    """
    sub_img, land_marks = sub_img_mark(db, data_path, regf_path, sub, slice_)
    marks = land_marks.values.reshape((-1, 2))
    mark_name = land_marks.index.values.reshape((-1, 2))
    n_marks = marks.shape[0]
    
    root = tkinter.Tk()
    root.wm_title("Subject %s Slice %s" % (sub, slice_))

    root.image = sub_img
    fig = plt.figure(figsize=(8, 5))
    ax = fig.add_subplot(111)
    im = ax.imshow(root.image)
    for i in range(n_marks):
        ix = marks[i, 0]
        iy = marks[i, 1]
        print('%s: %s, %s: %s' % (mark_name[i, 0], ix, mark_name[i, 1], iy))
        ax.plot(ix,iy, marker='o', markersize=8, markerfacecolor=(1, 1, 1, 0.1),markeredgewidth=1.5, markeredgecolor='r')
    plt.show()
    # canvas = FigureCanvasTkAgg(fig, master=root)
    # canvas.draw()
    # canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)
    #
    # toolbar = NavigationToolbar2Tk(canvas, root)
    # toolbar.update()
    # canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

In [43]:
from ipywidgets import interact_manual
import ipywidgets as widgets

interact_manual(disp_mark, db=widgets.Dropdown(
    options=[('Four chamber (2)', 2), ('Short axis (17)',17)],
    value=2,
    description='Database',
    disabled=False,), 
                data_path = input('Data full path (e.g. D:/ML_data/PAH/PAH1DB2.npy): '), 
                regf_path = input('Registration file full path (e.g. D:/ML_data/PAH/regMRI4ChFull.xlsx): '), 
                sub=widgets.IntText(
                    value=1005,
                    description='Subject ID (e.g. 1005):',
                    disabled=False),
                slice_=widgets.IntSlider(
                    value=0,
                    min=0,
                    max=19,
                    step=1,
                    description='Slice (time point, 0-19):',
                    disabled=False,
                    continuous_update=True,
                    orientation='horizontal',
                    readout=True,
                    readout_format='d'), 
               )

Data full path (e.g. D:/ML_data/PAH/PAH1DB2.npy): D:/ML_data/PAH/PAH1DB2.npy
Registration file full path (e.g. D:/ML_data/PAH/regMRI4ChFull.xlsx): D:/ML_data/PAH/regMRI4ChFull.xlsx


interactive(children=(Dropdown(description='Database', options=(('Four chamber (2)', 2), ('Short axis (17)', 1…

<function __main__.disp_mark(db, data_path, regf_path, sub, slice_)>

### Interactive Marking (Get coords manually)

In [45]:
def onclick(event):
    """Get the coords and draw a circle at the postition of a single click
    on the image

    Args:
        event: interaction event, e.g. click

    Returns:
        click coords
    """
    global ix, iy
    ix, iy = event.xdata, event.ydata
    # print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
    #       ('double' if event.dblclick else 'single', event.button,
    #        event.x, event.y, event.xdata, event.ydata))
    print('%s click: button=%d, x=%f, y=%f' %
          ('double' if event.dblclick else 'single', 
           event.button, event.xdata, event.ydata))    
    # ax = fig.add_subplot(111)
    ax.plot(ix,iy, marker='o', markersize=8, markerfacecolor=(1, 1, 1, 0.1),markeredgewidth=1.5, markeredgecolor='r')
    canvas.draw()

    global coords
    coords.append((ix, iy))

#     if len(coords) == 2:
#         fig.canvas.mpl_disconnect(cid)

    return coords


def _quit():
    root.quit()     # stops mainloop
    root.destroy()  # this is necessary on Windows to prevent
                    # Fatal Python Error: PyEval_RestoreThread: NULL tstate


def hand_mark(db, data_path, regf_path, sub, slice_):
    """Marking landmarks manually on a CMR slice

    Args:
        db (int): data base id, 2 for 4 chamber and 17 for short axis
        data_path (str): full path to data file
        regf_path (str): full path to landmark location .xlsx file
        sub (int): subject index
        slice_ (int): slice index

    Returns:

    """
    sub_img, land_marks = sub_img_mark(db, data_path, regf_path, sub, slice_)
    global root, fig, im, ax, canvas, coords
    root = tkinter.Tk()
    root.wm_title("Subject %s Slice %s" % (sub, slice_))

    # fig = Figure(figsize=(5, 4), dpi=100)
    # t = np.arange(0, 3, .01)
    # fig.add_subplot(111).plot(t, 2 * np.sin(2 * np.pi * t))

    # root.image = plt.imread('index.png')
    # root.image = plt.imshow(sub_img, cmap='gray', vmin=0, vmax=255)
    root.image = sub_img
    fig = plt.figure(figsize=(8, 5))
    ax = fig.add_subplot(111)
    im = ax.imshow(root.image)

    canvas = FigureCanvasTkAgg(fig, master=root)  # A tk.DrawingArea.
    canvas.draw()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

    toolbar = NavigationToolbar2Tk(canvas, root)
    toolbar.update()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

    coords = []
    
    cid = fig.canvas.mpl_connect('button_press_event', onclick)

    button = tkinter.Button(master=root, text="Quit", command=_quit)
    button.pack(side=tkinter.TOP)

    tkinter.mainloop()

In [47]:
from ipywidgets import interact_manual
import ipywidgets as widgets

interact_manual(hand_mark, db=widgets.Dropdown(
    options=[('Four chamber (2)', 2), ('Short axis (17)',17)],
    value=2,
    description='Database',
    disabled=False,), 
                data_path = input('Data full path (e.g. D:/ML_data/PAH/PAH1DB2.npy): '), 
                regf_path = input('Registration file full path (e.g. D:/ML_data/PAH/regMRI4ChFull.xlsx): '), 
                sub=widgets.IntText(
                    value=1005,
                    description='Subject ID (e.g. 1005):',
                    disabled=False),
                slice_=widgets.IntSlider(
                    value=0,
                    min=0,
                    max=19,
                    step=1,
                    description='Slice (time point, 0-19):',
                    disabled=False,
                    continuous_update=True,
                    orientation='horizontal',
                    readout=True,
                    readout_format='d'), 
        )

Data full path (e.g. D:/ML_data/PAH/PAH1DB2.npy): D:/ML_data/PAH/PAH1DB2.npy
Registration file full path (e.g. D:/ML_data/PAH/regMRI4ChFull.xlsx): D:/ML_data/PAH/regMRI4ChFull.xlsx


interactive(children=(Dropdown(description='Database', options=(('Four chamber (2)', 2), ('Short axis (17)', 1…

<function __main__.hand_mark(db, data_path, regf_path, sub, slice_)>

### Update coords and save to file

In [48]:
def update_coords(db, regf_path, sub, mark_names, mark_values, out_dir):
    """Update landmark location coords for a subject and save to the given directory
    
    Args:
        db (int): data base id, 2 for 4 chamber and 17 for short axis
        regf_path (str): full path to landmark location .xlsx file
        sub (int): subject index
        mark_names (str list): names of landmark to be updated
        mark_values (float list): new values of landmark coords.
        out_dir (str): directory for saving output data

    """
    mark_names = mark_names.split(',')
    mark_values = mark_values.split(',')
    n_marks = len(mark_names)
    check_dir(out_dir)
    if n_marks == len(mark_values):   
        reg_df = load_landmark(regf_path, db)
        sub_idx = np.where(reg_df['ID'] == sub)[0][0]
        for i in range(len(mark_names)):
            reg_df.loc[sub_idx, mark_names[i]] = int(mark_values[i])
        out_fname = 'new_regDB%s.csv' % db
        reg_df.to_csv(os.path.join(out_dir, out_fname))
        print('Completed, new landmark file %s saved to %s' % (out_fname, out_dir))
    else:
        print('Number of landmark names and values are not consistant!!')
        sys.exit()

In [49]:
from ipywidgets import interact_manual
import ipywidgets as widgets

interact_manual(update_coords, db=widgets.Dropdown(
    options=[('Four chamber (2)', 2), ('Short axis (17)',17)],
    value=2,
    description='Database',
    disabled=False,), 
                regf_path = input('Registration file full path (e.g. D:/ML_data/PAH/regMRI4ChFull.xlsx): '), 
                sub=widgets.IntText(
                    value=1005,
                    description='Subject ID (e.g. 1005):',
                    disabled=False),
                mark_names=input('Landmark Names (separate by comma, e.g. Spinal X,Spinal Y): '),
                mark_values=input('New landmark values (separate by comma): '),
                out_dir = input('Where to save the output (e.g. D:/ML_data/PAH/DB2): '),
        )

Registration file full path (e.g. D:/ML_data/PAH/regMRI4ChFull.xlsx): D:/ML_data/PAH/regMRI4ChFull.xlsx
Landmark Names (separate by comma, e.g. Spinal X,Spinal Y): Spinal X,Spinal Y
New landmark values (separate by comma): 10, 10
Where to save the output (e.g. D:/ML_data/PAH/DB2): D:/ML_data/PAH/DB2


interactive(children=(Dropdown(description='Database', options=(('Four chamber (2)', 2), ('Short axis (17)', 1…

<function __main__.update_coords(db, regf_path, sub, mark_names, mark_values, out_dir)>